# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
#!pip install geopandas

In [5]:
# Dependencies and Setup
import hvplot.pandas
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt
import holoviews as hv


# Import API key
from api_keys import geoapify_key

In [9]:
#!pip install --upgrade hvplot holoviews

In [25]:
!pip install geoviews

  Obtaining dependency information for geoviews from https://files.pythonhosted.org/packages/72/10/81aaca3edf7d7bfb14a9a17b5effaeb9dcba54ba9703b392bb17241e45ea/geoviews-1.11.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for cartopy>=0.18.0 from https://files.pythonhosted.org/packages/41/f4/edfba7b0768930486d4f434183a500de75826aba82e6ffd4884ae2b410b9/Cartopy-0.22.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     -------- ------------------------------- 10.2/46.5 kB ? eta -:--:--
     -------------------------------------- 46.5/46.5 kB 772.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/511.2 kB ? eta -:--:--
   ------------ --------------------------- 163.8/511.2 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------  501.8/511.2 kB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 511.2/511.2 kB 5.3 MB/s eta 0:00:00
   --------------------------------------

In [26]:
!pip install cartopy


In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,65.07,61.0,4.0,6.42,NZ,1.706485e+09
1,1,adamstown,-25.0660,-130.1015,76.21,76.0,100.0,9.91,PN,1.706485e+09
2,2,albany,42.6001,-73.9662,34.36,85.0,100.0,1.01,US,1.706484e+09
3,3,isafjordur,66.0755,-23.1240,21.61,90.0,100.0,4.00,IS,1.706485e+09
4,4,pont rouge,48.8833,-72.0825,30.49,96.0,98.0,3.53,CA,1.706485e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# from holoviews.element.tiles import EsriImagery
# EsriImagery()

In [31]:
# Create a scatter plot with hvplot (code sourced from https://holoviz.org/tutorial/Composing_Plots.html, 1/27/24)

scatter_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity'], title='City Humidity Map', line_color='black')


# # Display the plot
scatter_plot


:Points   [Lng,Lat]   (Humidity,City)

In [23]:
%%capture --no-display

# Configure the map plot. c is color, but we can also set to a variable. geo=True indicates plot is geospatial
humidity_scatter_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity'], title='City Humidity Map', tiles='EsriImagery', line_color='black')


#Display the map
humidity_scatter_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:

# Narrow down cities that fit criteria and drop any results with null values
city_data_locations = city_data_df[(city_data_df['Max Temp'] > 65) & (city_data_df['Humidity'] < 65) & (city_data_df['Wind Speed'] < 15) 
            & (city_data_df['Cloudiness'] < 45)]

# Drop any rows with null values
city_data_locations.dropna(axis=0)

# Display sample data
city_data_locations.count()

City_ID       49
City          49
Lat           49
Lng           49
Max Temp      49
Humidity      49
Cloudiness    49
Wind Speed    49
Country       49
Date          49
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ('')

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,papatowai,-46.5619,169.4708,65.07,61.0,4.0,6.42,NZ,1.706485e+09,
1,1,adamstown,-25.0660,-130.1015,76.21,76.0,100.0,9.91,PN,1.706485e+09,
2,2,albany,42.6001,-73.9662,34.36,85.0,100.0,1.01,US,1.706484e+09,
3,3,isafjordur,66.0755,-23.1240,21.61,90.0,100.0,4.00,IS,1.706485e+09,
4,4,pont rouge,48.8833,-72.0825,30.49,96.0,98.0,3.53,CA,1.706485e+09,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    
# Set parameters to search for a hotel
#ChatGPT 2024, personal communication, January 27,2024
    radius = 10000
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    params = {
    "apiKey":geoapify_key,
    "filters":filters,
    "bias":bias,
    "type":"accommodation.hotel"
        
    }
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #ChatGPT 2024, personal communication, January 27,2024
   
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
      
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #print to debug
    #print("API Response:", name_address) #  -- this is no good I get this output
   # API Response: {'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
    #tiksi - nearest hotel: No hotel found
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
adamstown - nearest hotel: Chez Bianca & Benoit
albany - nearest hotel: Tru
isafjordur - nearest hotel: Hótel Horn
pont rouge - nearest hotel: Auberge La Diligence
susuman - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
harper - nearest hotel: hôtel le rochet
port-aux-francais - nearest hotel: Keravel
talnakh - nearest hotel: Талнах
dunedin - nearest hotel: Scenic Hotel Dunedin City
morada nova - nearest hotel: Casa Maris Ambrosina De Lims
grytviken - nearest hotel: Shorty's Motel
hermanus - nearest hotel: Aloe guest house
antsiranana - nearest hotel: Central
ghat - nearest hotel: فندق تاسيلي
afaahiti - nearest hotel: Omati Lodge
cargados carajos - nearest hotel: No hotel found
chirilagua - nearest hotel: Hotel Martinez
bethel - nearest hotel: Hampton Inn Danbury
puerto natales - nearest hotel: Dorotea Patagonia Hostel
west island - nearest hotel: Cocos Village Bungalows
pha

geraldton - nearest hotel: Ocean Centre Hotel
al majaridah - nearest hotel: The Distinctive Palace
souillac - nearest hotel: Hotel SkylineInn
urengoy - nearest hotel: Геолог
tuy hoa - nearest hotel: Vinh Tuan Hotel
akureyri - nearest hotel: Hótel Norðurland
tadine - nearest hotel: Hôtel Nengoné
aqadyr - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
wynyard - nearest hotel: Wynyard Hotel
pustoshka - nearest hotel: Гостиница Юбилейная
bubaque - nearest hotel: Chez Julio
vrede - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
tevriz - nearest hotel: Иртыш
la rioja - nearest hotel: No hotel found
moussoro - nearest hotel: Guesthouse دار الضيافة
arilje - nearest hotel: Hotel Mlinarev San
coquimbo - nearest hotel: Hotel Iberia
castlegar - nearest hotel: Sandman Hotel Castlegar
keetmanshoop - nearest hotel: Canyon Hotel
jalai nur - nearest hotel: 满洲里福门客栈
fort bragg - nearest hotel: Airborne Inn Lodging
kununurra - nea

port alfred - nearest hotel: No hotel found
bikenibeu village - nearest hotel: Otintaai Hotel
karungdong - nearest hotel: Embassy Hotel
berezovyy - nearest hotel: Пастораль
puerto baquerizo moreno - nearest hotel: Dory's House
tondano - nearest hotel: Hotel Tou Dano
floriano - nearest hotel: Pousada Guadalupe
kapuskasing - nearest hotel: Travelodge
port-gentil - nearest hotel: Tara-Mé
kuvandyk - nearest hotel: Отдыхающая локомотивных бригад
pingliang - nearest hotel: 平凉新都酒店
marsa alam - nearest hotel: Blue House Hotel
savantvadi - nearest hotel: Krezy Foods
barnoldswick - nearest hotel: The Craven Heifer Inn
totness - nearest hotel: Luxor Hotel
greymouth - nearest hotel: Recreation Hotel
caucaia - nearest hotel: Motel Kelps
nimbahera - nearest hotel: Hotel Meera
myitkyina - nearest hotel: New Light Hotel
al khums - nearest hotel: فندق الخمس
touros - nearest hotel: Pousada Atlântico
klaksvik - nearest hotel: Hotel Klaksvik
tarko-sale - nearest hotel: Вынгапур
lautoka - nearest hotel: Ca

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,papatowai,-46.5619,169.4708,65.07,61.0,4.0,6.42,NZ,1.706485e+09,Catlins Woodstock Lodge & Camp
1,1,adamstown,-25.0660,-130.1015,76.21,76.0,100.0,9.91,PN,1.706485e+09,Chez Bianca & Benoit
2,2,albany,42.6001,-73.9662,34.36,85.0,100.0,1.01,US,1.706484e+09,Tru
3,3,isafjordur,66.0755,-23.1240,21.61,90.0,100.0,4.00,IS,1.706485e+09,Hótel Horn
4,4,pont rouge,48.8833,-72.0825,30.49,96.0,98.0,3.53,CA,1.706485e+09,Auberge La Diligence


In [18]:
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,papatowai,-46.5619,169.4708,65.07,61.0,4.0,6.42,NZ,1.706485e+09,Catlins Woodstock Lodge & Camp
1,1,adamstown,-25.0660,-130.1015,76.21,76.0,100.0,9.91,PN,1.706485e+09,Chez Bianca & Benoit
2,2,albany,42.6001,-73.9662,34.36,85.0,100.0,1.01,US,1.706484e+09,Tru
3,3,isafjordur,66.0755,-23.1240,21.61,90.0,100.0,4.00,IS,1.706485e+09,Hótel Horn
4,4,pont rouge,48.8833,-72.0825,30.49,96.0,98.0,3.53,CA,1.706485e+09,Auberge La Diligence


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot

scatter_plot_2 = hotel_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity', 'Country', 'Hotel Name'], title='City Humidity Map', line_color='black')


# Display the map
scatter_plot_2

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)